In [ ]:
import pathlib
import os
from qibolab.paths import qibolab_folder
from qibolab import Platform

script_folder = pathlib.Path(os.path.abspath(''))
diagnostics_settings = script_folder / "diagnostics.yml"
runcard = qibolab_folder / 'runcards' / 'qw5q.yml' 

# Create a platform; connect and configure it
platform = Platform('multiqubit', runcard)
platform.connect()
platform.setup()

In [ ]:
import diagnostics_5q

ds = diagnostics_5q.Diagnostics(platform, diagnostics_settings)

Flux Spectroscopy

In [ ]:
for qubit in range(6):
    for fluxline in range(5):
        ds.run_resonator_spectroscopy_flux(qubit, fluxline)

In [ ]:
for n in range(16, 6*5):
    qubit = n // 5
    fluxline = n % 5
    print(f"scan:{n} qubit: {qubit} fluxline: {fluxline}\n")
    ds.run_resonator_spectroscopy_flux(qubit, fluxline)

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np 
import os
from qibolab.paths import qibolab_folder
files = []
base_folder = qibolab_folder / 'calibration' / 'data'/ 'quantify' / '20220626' # Replace with date!

for experiment_folder in os.listdir(base_folder):
    file_path = base_folder / experiment_folder / 'dataset.hdf5'
    files.append(str(file_path))

l = len(files)
l.sort()

plt.subplots(figsize=(40,40))

for n, file in enumerate(files):
    f = h5py.File(file)
    xlen = f.attrs['xlen']
    ylen = f.attrs['ylen']
    x = f['x0'][:xlen] + 20e6
    y = f['x1'][:].reshape(xlen, ylen)[:,0]
    z = np.flipud(f['y0'][:].reshape(f.attrs['ylen'], f.attrs['xlen']))
    qubit = n // 5
    fluxline = n % 5
    plt.subplot(5,6, qubit+1 + fluxline*(6))
    plt.imshow(z, extent=[min(x),max(x),min(y),max(y)], aspect="auto")


plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
cax = plt.axes([0.85, 0.1, 0.015, 0.8])
plt.colorbar(cax=cax)


plt.show()

Single Qubit Spectroscopies & Rabi

In [ ]:
qubit = 5

In [ ]:
resonator_freq, avg_min_voltage, max_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)

In [ ]:
qubit_freq, max_ro_voltage, smooth_dataset, dataset = ds.run_qubit_spectroscopy(qubit)

In [ ]:
dataset, pi_pulse_duration, pi_pulse_amplitude, rabi_oscillations_pi_pulse_min_voltage, t1 = ds.run_rabi_pulse_length(qubit)

Spectroscopies Loop

In [ ]:
for qubit in range(5):
    resonator_freq, avg_max_voltage, min_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)

for qubit in range(5):
    qubit_freq, max_ro_voltage, smooth_dataset, dataset = ds.run_qubit_spectroscopy(qubit)

In [ ]:
while True:
    resonator_freq, avg_min_voltage, max_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)

In [ ]:
"""
files = []
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-115433-962-82dd8b-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-124606-847-d03c47-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-133808-534-7bdec2-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-143125-212-35e1bb-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-152641-950-e4bc4b-matrix3\\dataset.hdf5')
"""